##ref code!!!:
<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/keras/restorers/Train_Zero_DCE_Restorers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{restorers-zero-dce-train} -->

ref code : https://colab.research.google.com/github/wandb/examples/blob/master/colabs/keras/restorers/Train_Zero_DCE_Restorers.ipynb

#Pls read : code Train_Zero_DCE
*   เป็นโค้ดที่ดึงค่า  weight  มาจาก Restorers ใน https://github.com/TrystAI/restorers
*   ในกรณีที่ต้องการเขียนและเทรนโมเดลด้วยตัวเอง สามารถทดลอง original code : https://colab.research.google.com/github/wandb/examples/blob/master/colabs/keras/Zero_DCE_Colab_Keras(video).ipynb

เนื่องจากเราต้องการทดลองความสามารถของ Zero_DCE จึงไม่ได้มีการปรับ  config เพื่อเพิ่มความสามารถในการเทรนโมเดล


## ดึง  library และ โครงสร้าง model จาก  Github

In [ ]:
!pip install -q --upgrade pip setuptools
!pip install git+https://github.com/soumik12345/restorers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 14.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
  Cloning https://github.com/soumik12345/restorers.git to /tmp/pip-req-build-074jdbcp
  Running command git clone --filter=blob:none --quiet https://github.com/soumik12345/restorers.git /tmp/pip-req-build-074jdbcp
  Resolved https://github.com/soumik12345/restorers.git to commit f84b63af3f6b5fc117e2a34eac00880180f8aa0e
  Installing build dependencies ... done
  Getting require

In [ ]:
import os
from glob import glob

import tensorflow as tf

import wandb
# import the wandb callbacks for keras
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

## log in wandb + Import dataset
*   สามารถดู wandb_api_key ของตัวเอง ได้ที่ https://wandb.ai/authorize
*   ดึง Lol dataset จาก artifact_address
*   ตั้งเงื่อนไขสำหรับชุดข้อมูล

In [ ]:
wandb.init(project="low-light-enhancement", job_type="train")


def load_data(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(
        images=image,
        size=[256, 256]
    )
    image = image / ((2 ** 8) - 1)
    return image


def data_generator(low_light_images):
    dataset = tf.data.Dataset.from_tensor_slices((low_light_images))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(8, drop_remainder=True)
    return dataset


artifact = wandb.use_artifact("ml-colabs/dataset/LoL:v0", type='dataset')
artifact_dir = artifact.download()

train_low_light_images = sorted(glob(os.path.join(artifact_dir, "our485", "low", "*")))
num_train_images = int((1 - 0.2) * len(train_low_light_images))
val_low_light_images = train_low_light_images[num_train_images:]
train_low_light_images = train_low_light_images[:num_train_images]

train_dataset = data_generator(train_low_light_images)
val_dataset = data_generator(val_low_light_images)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Downloading large artifact LoL:v0, 331.95MB. 1003 files... 
wandb:   1003 of 1003 files downloaded.  
Done. 0:0:17.3


## Import MirNetv2 from restorers
*   ตั้งเงื่อนไขสำหรับ model

In [ ]:
from restorers.model.zero_dce import ZeroDCE
model = ZeroDCE(
    num_intermediate_filters=32, # number of filters in the intermediate convolutional layers
    num_iterations=8, # number of iterations of enhancement
    decoder_channel_factor=1 # factor by which number filters in the decoder of deep curve estimation layer is multiplied
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    weight_exposure_loss=1.0, # weight of the exposure control loss
    weight_color_constancy_loss=0.5, # weight of the color constancy loss
    weight_illumination_smoothness_loss=20, # weight of the illumination smoothness loss
)

##Train model + call back  ผลลัพธ์จากการฝึก

In [ ]:
callbacks = [
    WandbMetricsLogger(log_freq="batch"),
    WandbModelCheckpoint(
        filepath="checkpoint",
        monitor="val_loss",
        save_best_only=False,
        save_weights_only=False,
        initial_value_threshold=None,
    )
]

model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=callbacks,
)

Epoch 1/100
 6/48 [==>...........................] - ETA: 5s - total_loss: 0.5595 - illumination_smoothness_loss: 0.0128 - spatial_constancy_loss: 7.4495e-05 - color_constancy: 4.6791e-04 - exposure_control_loss: 0.3040

48/48 [==============================] - ETA: 0s - total_loss: 0.5524 - illumination_smoothness_loss: 0.0130 - spatial_constancy_loss: 1.6746e-04 - color_constancy: 6.5400e-04 - exposure_control_loss: 0.2913

wandb: Adding directory to artifact (./checkpoint)... Done. 0.0s


48/48 [==============================] - 19s 213ms/step - total_loss: 0.5500 - illumination_smoothness_loss: 0.0129 - spatial_constancy_loss: 1.6446e-04 - color_constancy: 6.4408e-04 - exposure_control_loss: 0.2918 - val_total_loss: 0.3829 - val_illumination_smoothness_loss: 0.0043 - val_spatial_constancy_loss: 6.2235e-05 - val_color_constancy: 6.2323e-05 - val_exposure_control_loss: 0.2960
Epoch 2/100
48/48 [==============================] - ETA: 0s - total_loss: 0.4168 - illumination_smoothness_loss: 0.0063 - spatial_constancy_loss: 1.1548e-04 - color_constancy: 6.4341e-04 - exposure_control_loss: 0.2910

wandb: Adding directory to artifact (./checkpoint)... Done. 0.0s


...

48/48 [==============================] - 9s 185ms/step - total_loss: 0.2203 - illumination_smoothness_loss: 1.0516e-04 - spatial_constancy_loss: 0.0503 - color_constancy: 0.0047 - exposure_control_loss: 0.1656 - val_total_loss: 0.2239 - val_illumination_smoothness_loss: 7.5830e-05 - val_spatial_constancy_loss: 0.0468 - val_color_constancy: 9.8331e-04 - val_exposure_control_loss: 0.1751
Epoch 99/100
48/48 [==============================] - ETA: 0s - total_loss: 0.2200 - illumination_smoothness_loss: 1.0514e-04 - spatial_constancy_loss: 0.0507 - color_constancy: 0.0048 - exposure_control_loss: 0.1648

wandb: Adding directory to artifact (./checkpoint)... Done. 0.0s


48/48 [==============================] - 9s 179ms/step - total_loss: 0.2203 - illumination_smoothness_loss: 1.0416e-04 - spatial_constancy_loss: 0.0502 - color_constancy: 0.0047 - exposure_control_loss: 0.1657 - val_total_loss: 0.2239 - val_illumination_smoothness_loss: 7.9487e-05 - val_spatial_constancy_loss: 0.0469 - val_color_constancy: 9.8550e-04 - val_exposure_control_loss: 0.1749
Epoch 100/100
48/48 [==============================] - ETA: 0s - total_loss: 0.2200 - illumination_smoothness_loss: 1.0564e-04 - spatial_constancy_loss: 0.0507 - color_constancy: 0.0048 - exposure_control_loss: 0.1647

wandb: Adding directory to artifact (./checkpoint)... Done. 0.0s


48/48 [==============================] - 8s 172ms/step - total_loss: 0.2203 - illumination_smoothness_loss: 1.0461e-04 - spatial_constancy_loss: 0.0503 - color_constancy: 0.0047 - exposure_control_loss: 0.1656 - val_total_loss: 0.2239 - val_illumination_smoothness_loss: 7.7164e-05 - val_spatial_constancy_loss: 0.0468 - val_color_constancy: 9.8159e-04 - val_exposure_control_loss: 0.1751


In [ ]:
wandb.finish()

batch/batch_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
batch/color_constancy,▁▁▁▂▂▆█▅▆█▆█▅▆█▅▆█▅▆▅▆█▅▆█▅▆█▅█▅▆█▅▆█▅▆█
batch/exposure_control_loss,█▇▇█▅▂▄▁▂▄▂▄▁▂▄▁▂▄▁▂▁▂▄▁▂▄▁▂▄▁▄▁▂▄▁▂▄▁▂▄
batch/illumination_smoothness_loss,█▆▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/spatial_constancy_loss,▁▁▁▁▂█▄▇█▄█▄██▄██▄████▄██▄██▄█▄██▄██▄██▄
batch/total_loss,█▇▅▅▃▂▃▁▂▃▂▃▁▂▃▁▂▃▁▂▁▂▂▁▂▂▁▂▂▁▂▁▂▂▁▂▂▁▂▂
epoch/color_constancy,▁▁▁▁▃███████████████████████████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/exposure_control_loss,██▇▇▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/illumination_smoothness_loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
